# Shot mesh visualizations for additional NBA teams
Applying the same gravitational potential principle for other NBA team shot meshes.

In [43]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import normalize

def createShotMesh(data, fp, team, season):

    # Read point masses from a CSV file
    df = pd.read_csv(data)

    # Extract x, y, and m values
    masses = df[['x', 'y', 'm']].values  # Convert to NumPy array

    # Gravitational constant (arbitrary scaling factor for visualization)
    G = 1

    # Create a grid of points
    x_min, x_max = -250, 250
    y_min, y_max = -42, 458
    x, y = np.linspace(x_min, x_max, 101), np.linspace(y_min, y_max, 101)
    X, Y = np.meshgrid(x, y)

    # Initialize the potential energy array
    U = np.zeros(X.shape)

    # Calculate the potential energy at each grid point
    for mx, my, m in masses:
        dx = X - mx
        dy = Y - my
        r = np.sqrt(dx**2 + dy**2)
        r = np.where(r != 0, r, np.inf)  # Avoid division by zero
        U += G * m / r

    # Scale the potential energy for better visualization
    U_scaled = (U - U.min() + 1) / (U.max() - U.min())
    U_scaled = np.log1p(U_scaled)  # Log scaling for better contrast
    U_scaled = np.log1p(np.sqrt(U_scaled))  # Log scaling for better contrast

    # Create the 3D interactive plot using Plotly
    fig = go.Figure()

    # Add the surface for potential energy
    fig.add_trace(go.Surface(z=U_scaled, x=X, y=Y, colorscale='inferno', colorbar=dict(title="Shot Potential")))

    # Update layout for better visualization
    fig.update_layout(
        title=f"{team} {season} Season - Shot Potential Energy",
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Shot Potential'
        ),
        margin=dict(l=0, r=0, b=0, t=40),
    )

    # Show the interactive plot
    fig.write_html(fp)

In [6]:
createShotMesh('BOS_shotgravity.csv', 'shotgravityfunction.html', 'Boston Celtics', '2023-24')

# Clean data for shot mesh function

In [14]:
import pandas as pd

fp = '../../data/processed/S2324/S2324_all_play-by-play.csv'

def createShotMass(fp, teamTricode):

    df = pd.read_csv(fp)

    df = df[(df['teamTricode'] == teamTricode) & (df['isFieldGoal'] == 1)]

    cols = ['xLegacy', 'yLegacy', 'scoreVal']
    df = df[cols].reset_index(drop=True)

    df['m'] = df['scoreVal'] - 1
    df = df.rename(columns={'xLegacy': 'x', 'yLegacy': 'y'})
    df = df.drop(columns=['scoreVal'])
    df.to_csv(f"shotmass/{teamTricode}_shotmass.csv", index=False)

createShotMass(fp, 'BOS')

## Create all shotmass files for each team

In [15]:
fp = '../../data/processed/S2324/S2324_all_play-by-play.csv'
df = pd.read_csv(fp)
teams = df['teamTricode'].unique()
# Remove nan from teams
teams = teams[~pd.isnull(teams)]

for team in teams:
    createShotMass(fp, team)

## Turn all shotmass files into potential energy visualizations

In [44]:
# For every file in a given directory, create a shot mesh
import os
import pandas as pd

directory = 'shotmass'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        team = filename.split('_')[0]
        createShotMesh(f"{directory}/{filename}", f"shotmesh_v0/{team}_shotmesh.html", team, '2023-24')
    else:
        continue

